In [1]:
import requests
import pandas as pd
from anytree import Node, RenderTree
from functions import *
from Reddit_functions import *

In [2]:
with open('Authentication/database_uri.txt', 'r', encoding="utf8") as f:
    uri = f.read()

In [3]:
with open('Authentication/reddit_credentials.txt', 'r', encoding="utf8") as f:
    credentials = f.read()
    
credentials = credentials.split('\n')

In [4]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(credentials[0], credentials[1])

In [5]:
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': credentials[2],
        'password': credentials[3]}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

## Get all Reddit Posts

In [6]:
getRedditPosts('Singapore')

Post ID: u2tc1u Time: 2022-04-13 23:47:48
Severe Covid-19 symptoms 5 times more likely in patients who got Sinovac rather than Pfizer jab: Study 

Post ID: u2sw33 Time: 2022-04-13 23:27:41
10 Amazing and Shocking Facts About Jungkook of BTS. K-pop Lover's 

Post ID: u2si3g Time: 2022-04-13 23:10:10
Woman With End-Stage Cancer Volunteers At Thomson Hospice, Wants To Help Others While Able 

Post ID: u2scj1 Time: 2022-04-13 23:03:21
A leader without followers is simply a man taking a walk 

Post ID: u2sbf7 Time: 2022-04-13 23:02:01
laptop 

Post ID: u2s8ff Time: 2022-04-13 22:58:45
About 200 ships supplied with contaminated bunker fuel in Port of Singapore: MPA 

Post ID: u2rrib Time: 2022-04-13 22:36:44
Warning letters issued to 3 businesses over complaints of unfinished renovation works 

Post ID: u2qwyg Time: 2022-04-13 21:57:09
Coldplay, BTS, and mental health as the next pandemic | Off Duty with Minister Ong Ye Kung 

Post ID: u2qnji Time: 2022-04-13 21:44:42
Divers wanted: New citi

## Get Comments of Individual Posts

In [7]:
# while the token is valid (~2 hours) we just add headers=headers to our requests
res = requests.get('https://oauth.reddit.com/r/singapore/comments/rmqevj', headers=headers)
# res = requests.get('https://oauth.reddit.com/r/singapore/comments/r08nub', headers=headers)

postID = 'rmqevj'
conversation_dict, tree = createTree(res, postID)

-----About the post-----
No. of Upvotes: 1871
Upvote Ratio: 0.87 

User: hpr2kav Time: 2021-12-24 08:55:24
In reply to: rmqevj
All I know is - anyone trying to pull a fast one and lie in parliament from now on will think trice,  and that's a good thing for all of us. 

User: hra9zzo
Time: 2022-01-05 08:14:35
In reply to: hpr2kav
Why does she think it is OK to lie about such things? Sense of impunity? 

User: hpob7bq Time: 2021-12-23 20:17:48
In reply to: rmqevj
A lie was told in parliament that affects the following:
1) women’s rights 
2) apparently now mental illness rights
3) the way Opposition finds candidates
4) the way opposition handles rogue MPs
5) the way opposition works internally and at the top
6) national sovereignty (cannot taint the whiter than white image even when this tainting is done by a light blue) other country still watching. This is like Hawkeye, he cannot miss if not he’s just another guy with a bow and arrow.
7) you don’t lie in parliament. 

Doing this doesn’t

In [8]:
df = processRedditDataframe(conversation_dict, postID)
df['url'] = df['comment'].apply(lambda x: getLinks(x))
df['link_title'] = df['url'].apply(lambda x: getURLfromList(x))

# df[df['id'] == 'hpoc9nq']
# df.to_csv('Datasets/reddit_data.csv', index=False)

df.head()

,user_name,id,timestamp,reply_to,comment,head_id,social_media,url,link_title
0,MapleViolet,hpr2kav,2021-12-24 08:55:24,rmqevj,All I know is - anyone trying to pull a fast o...,rmqevj,Reddit,,
1,HaddockFillet,hra9zzo,2022-01-05 08:14:35,hpr2kav,Why does she think it is OK to lie about such ...,rmqevj,Reddit,,
2,kuang89,hpob7bq,2021-12-23 20:17:48,rmqevj,A lie was told in parliament that affects the ...,rmqevj,Reddit,,
3,[deleted],hpprwf1,2021-12-24 03:06:33,hpob7bq,"I would like to add ""Public trust in a critica...",rmqevj,Reddit,,
4,kuang89,hprs02i,2021-12-24 12:23:36,hpprwf1,"Ya missed out on that, police public image als...",rmqevj,Reddit,,


In [9]:
for pre, fill, node in RenderTree(tree):
    print("%s%s" % (pre, node.name))

rmqevj
├── hpr2kav
│   └── hra9zzo
├── hpob7bq
│   ├── hpprwf1
│   │   └── hprs02i
│   └── hpr7oz0
│       └── hpry51w
├── hpntm2t
│   ├── hpnwekg
│   │   ├── hpnzb4r
│   │   │   └── hpo5qkg
│   │   │       └── hpokj06
│   │   │           ├── hpp0qgm
│   │   │           ├── hpoqxz5
│   │   │           └── hpraj0s
│   │   │               └── hpshgjh
│   │   ├── hpo2pl5
│   │   │   └── hpo7oxl
│   │   │       └── hpo7u0x
│   │   ├── hpo76sq
│   │   ├── hpo7l9d
│   │   ├── hpradku
│   │   │   ├── hps0rop
│   │   │   └── hprqpon
│   │   ├── hpoya8b
│   │   └── hpnwve6
│   │       └── hpnz3vc
│   │           ├── hpp0vzg
│   │           └── hpo6fwl
│   │               └── hpp7l4j
│   ├── hpo1src
│   │   ├── hpolpfk
│   │   ├── hpo33t8
│   │   │   ├── hpo4que
│   │   │   │   └── hpom3uq
│   │   │   │       └── hprayhb
│   │   │   │           └── hpruuh0
│   │   │   └── hpoq8hi
│   │   │       └── hppkszp
│   │   │           └── hpppv0m
│   │   └── hpo6898
│   │       └── hpshl11
│   ├── hpnx9

In [10]:
# df.iloc[42]

df[(40 <= df.index) & (df.index < 45)]

,user_name,id,timestamp,reply_to,comment,head_id,social_media,url,link_title
40,sec5,hpo6898,2021-12-23 19:16:42,hpo1src,I feel that WP are being sour grapes about thi...,rmqevj,Reddit,,
41,HazyNightz,hpshl11,2021-12-24 17:16:01,hpo6898,I believe PAP has also failed to investigate o...,rmqevj,Reddit,,
42,Durian881,hpnx97j,2021-12-23 17:13:29,hpntm2t,Well said. Maybe we should revisit the TraceTo...,rmqevj,Reddit,https://www.straitstimes.com/singapore/vivian-...,[www.straitstimes.com]
43,botsland,hpnxtjg,2021-12-23 17:21:09,hpnx97j,&gt;It's such a blatant lie. \n\nWe don't have...,rmqevj,Reddit,,
44,lockedoutofmyreddit,hpo0tg0,2021-12-23 18:02:58,hpnxtjg,Even if Vivian genuinely thought he was tellin...,rmqevj,Reddit,,
